Again, before start, let's import a few Python modules.

This time we will focus on a new module of `kongming.memory`.

In [1]:
from kongming import api, hv, memory 
from kongming.hv import helpers

d0 = hv.d0()

Like we did before, we generate some primitive random hypervectors.

In [2]:
primitives={chr(letter): d0.new_sparse_segmented_from_seed_word(api.MODEL_1M_10BIT, chr(letter)) for letter in range(ord('a'), ord('z')+1)}

str(primitives['a']), str(primitives['k'])

('a', 'k')

# Working with ChunkPickers

`memory.ChunkPickers`, as its name suggests, are pre-defined logic that selects and picks chunks, which are self-contained unit for a single meaningful unit. For example, we can pick constituents from a composite code representing a sequence, or we can pick nearby registered chunks from a given probe.

In [3]:
world = d0.new_sequence(1, primitives['w'], primitives['o'], primitives['r'], primitives['l'], primitives['d']).compose(1)

helpers.to_native_hbp(world.proto_load())

hint: SEQUENCE
model: MODEL_1M_10BIT
stable_hash: 7016629907439205749
sequence {
  seed: 1
  members {
    contains {
      model: MODEL_1M_10BIT
      sparse_segmented {
        seed_word: "w"
      }
    }
    contains {
      model: MODEL_1M_10BIT
      sparse_segmented {
        seed_word: "o"
      }
    }
    contains {
      model: MODEL_1M_10BIT
      sparse_segmented {
        seed_word: "r"
      }
    }
    contains {
      model: MODEL_1M_10BIT
      sparse_segmented {
        seed_word: "l"
      }
    }
    contains {
      model: MODEL_1M_10BIT
      sparse_segmented {
        seed_word: "d"
      }
    }
    checksum: 18116954493655476081
  }
}

The above `world` itself is a composed `hv.Sequence` instance, and you can think of it as a sequence code for the word `world`.

To recover individual members (`w`, `o`, `r`, `l` and `d`), we can use a instance `mp` of type `memory.sequence_members`.

In [4]:
mp = memory.sequence_members("", memory.with_semantic(world), 0, 0)

helpers.to_native_picker(mp.proto_load())

seq_members_of {
  sequence {
    chunk_literal {
      semantic {
        hint: SEQUENCE
        model: MODEL_1M_10BIT
        stable_hash: 7016629907439205749
        sequence {
          seed: 1
          members {
            contains {
              model: MODEL_1M_10BIT
              sparse_segmented {
                seed_word: "w"
              }
            }
            contains {
              model: MODEL_1M_10BIT
              sparse_segmented {
                seed_word: "o"
              }
            }
            contains {
              model: MODEL_1M_10BIT
              sparse_segmented {
                seed_word: "r"
              }
            }
            contains {
              model: MODEL_1M_10BIT
              sparse_segmented {
                seed_word: "l"
              }
            }
            contains {
              model: MODEL_1M_10BIT
              sparse_segmented {
                seed_word: "d"
              }
            }
            check

It has to be noted that the input for `memory.sequence_members` doesn't have to be the fully composed sequence code. Even with a flattened code (`world.core()`), the associative index should enable this code picker to do the right job. 

In [ ]:
mp0 = memory.sequence_members("", memory.with_semantic(world.core()), 0, 0)

helpers.to_native_picker(mp0.proto_load())

Notice the sequence codes from both `mp` and `mp1` are identical (from their `stable_hash`), and should return the same results.

# Interacts with memory

Once we have a good understanding of how hypervectors and their interactions, the next logical step is to store and retrieve them from a memory. 

Memory is a component that provides efficient storage and near-neighbor search capability for hypervectors. This is analogous to the vector database as discussed in industry. 